In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [5]:
from pyspark.conf import SparkConf
from pyspark import SparkContext, SQLContext

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql import Row

import pyspark

In [6]:
import nltk
# nltk.download('wordnet')

# SPACY INIT


In [ ]:
!pip install spacy

In [ ]:
!pip install spacy_langdetect

In [12]:
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language
from spacy.matcher import PhraseMatcher

In [13]:
%%time
nlp_basic = spacy.load("en_core_web_sm")

CPU times: user 527 ms, sys: 30.6 ms, total: 558 ms
Wall time: 674 ms


In [14]:
def create_lang_detector(nlp, name):
    return LanguageDetector()

In [15]:
%%time
Language.factory("language_detector", func=create_lang_detector)

CPU times: user 431 µs, sys: 0 ns, total: 431 µs
Wall time: 437 µs


<function __main__.create_lang_detector(nlp, name)>

In [16]:
%%time
nlp_basic.add_pipe('language_detector', last=True)

CPU times: user 1.34 ms, sys: 49 µs, total: 1.39 ms
Wall time: 1.39 ms


In [17]:
doc = nlp_basic("I really like you I really like you")

In [18]:
language = doc._.language.get('language')

In [19]:
language

'en'

## PYSPARK DF INIT

In [20]:
pyspark.__version__

'3.3.1'

In [21]:
master = "local"
master = master + "[*, 1000]"


conf = SparkConf().setAppName("klassifier").setMaster(master).setAll([#('spark.executor.memory',      '8G'), 
																			#("spark.dynamicAllocation.enabled",      "true"),
																			#("spark.dynamicAllocation.minExecutors", "1"),
																			#("spark.dynamicAllocation.maxExecutors", "1"),
																			#("spark.executor.instances",             "4"),
																			#('spark.executor.cores',                 '4'), 
																			#('spark.cores.max',                      '4'), 
																			#('spark.driver.memory',                  '8G'), 
																			#('spark.driver.maxResultSize',           '5G'),
																			#('spark.sql.execution.arrow.enabled',    arrowEnabled),
																			('spark.sql.execution.arrow.pyspark.enabled', "false"),
																			#('spark.logConf',                        'true'),
																			#('spark.default.parallelism',            1),
																			#('spark.sql.shuffle.partitions',         1),
																			('spark.eventLog.enabled',               'false')
																			])

In [22]:
sc = SparkContext.getOrCreate();

In [23]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [24]:
def readCSVFile(fileDelimeter, dataFileName, encoding = "utf-8") :

		from pyspark.storagelevel import StorageLevel
		return sqlContext.read.option("delimiter",
											fileDelimeter).option("encoding", 
											   						encoding).csv(dataFileName, 
																					header = True, 
																					inferSchema = True)

In [25]:
def convertToLowerCase(fullCozumData, featureColumnName):

    fullCozumData = fullCozumData.withColumn('lower_body', F.lower(F.col(featureColumnName)))
    fullCozumData = fullCozumData.drop(featureColumnName)
    fullCozumData = fullCozumData.withColumnRenamed('lower_body', featureColumnName)

    return fullCozumData     

### PYSPARK READ FILE

In [30]:
df = readCSVFile(",", "/content/drive/My Drive/AI/Tweets_airline_Complain_Upload_Ready2.csv", encoding = "utf-8")

In [31]:
df.show()

+--------------------+--------------------+
|      negativereason|                text|
+--------------------+--------------------+
|          Bad Flight|"@VirginAmerica i...|
|         Late Flight|@VirginAmerica SF...|
|          Bad Flight|@VirginAmerica  I...|
|Customer Service ...|@VirginAmerica yo...|
|Customer Service ...|@VirginAmerica st...|
|          Bad Flight|@VirginAmerica am...|
|Flight Booking Pr...|@VirginAmerica hi...|
|Customer Service ...|@VirginAmerica he...|
|Customer Service ...|@VirginAmerica aw...|
|Customer Service ...|@VirginAmerica Yo...|
|Flight Booking Pr...|@VirginAmerica He...|
|Flight Booking Pr...|@VirginAmerica hi...|
|Customer Service ...|@VirginAmerica I ...|
|Customer Service ...|@VirginAmerica he...|
|         Late Flight|@VirginAmerica Hi...|
|        Lost Luggage|@VirginAmerica ev...|
|Flight Attendant ...|@VirginAmerica yo...|
|Customer Service ...|@VirginAmerica wh...|
|Customer Service ...|@VirginAmerica wh...|
|         Late Flight|@VirginAme

In [32]:
featureColumnNames = ['text']

In [33]:
labelColumnName = 'negativereason'

In [34]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in featureColumnNames + [labelColumnName]]).show()


+----+--------------+
|text|negativereason|
+----+--------------+
|  52|             0|
+----+--------------+



In [35]:
df= df.select(featureColumnNames  + [labelColumnName]).replace('?', None).dropna(how='any') # clean null rows, this will be replaced to somewhere else.

In [36]:
df.show(5)

+--------------------+--------------------+
|                text|      negativereason|
+--------------------+--------------------+
|"@VirginAmerica i...|          Bad Flight|
|@VirginAmerica SF...|         Late Flight|
|@VirginAmerica  I...|          Bad Flight|
|@VirginAmerica yo...|Customer Service ...|
|@VirginAmerica st...|Customer Service ...|
+--------------------+--------------------+
only showing top 5 rows



In [37]:
df.dtypes

[('text', 'string'), ('negativereason', 'string')]

In [38]:
for column in featureColumnNames + [labelColumnName]:
    df = convertToLowerCase(df, column)

In [39]:
df.dtypes

[('text', 'string'), ('negativereason', 'string')]

In [40]:
df.show(5)

+--------------------+--------------------+
|                text|      negativereason|
+--------------------+--------------------+
|"@virginamerica i...|          bad flight|
|@virginamerica sf...|         late flight|
|@virginamerica  i...|          bad flight|
|@virginamerica yo...|customer service ...|
|@virginamerica st...|customer service ...|
+--------------------+--------------------+
only showing top 5 rows



In [41]:
from pyspark.ml.feature import StringIndexer

df = StringIndexer(
    inputCol= labelColumnName, 
    outputCol='label', 
    handleInvalid='keep').fit(df).transform(df)


df = df.drop(labelColumnName)
labelColumnName = 'label'

In [42]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [43]:
NLP = None
MATCHER = None

In [44]:
stop_words = set("""th nd rd st a b c d e f g h i j k l m n o p q r s t u v w x y z""".split())

In [45]:
def get_spacy(lang):
    global NLP
    if NLP is None:
        NLP = {}
    if lang not in NLP:

        if lang == 'en':
            nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
            nlp.Defaults.stop_words |= stop_words
        else:
            nlp = spacy.blank(lang)
            nlp.add_pipe("lemmatizer")
            nlp.initialize()

        NLP[lang] = nlp

    return NLP[lang]    

In [46]:
def get_matcher(terms):
    global MATCHER

    nlp = get_spacy('en')
    if MATCHER is None:
        MATCHER = PhraseMatcher(nlp.vocab, attr='LEMMA')
        patterns = [nlp(text) for text in terms]
        MATCHER.add("list: ", patterns)

    return MATCHER

In [47]:
%%time
def spacy_noun_lemmatize_matcher(lang, terms):
    def nounLemmatizerMatcher(text):
        nlp = get_spacy(lang)
        matcher = get_matcher(terms)
        result = []
        if (text is not None and len(text) > 0):
            doc = nlp(text)
            matches = matcher(doc)
            for match in matches:
                match_id, start, end = match
                tmp = doc[start:end].lemma_
                if tmp not in result :
                    result.append(tmp)    
            result = result + [token.lemma_ for token in doc if token.pos_ == "NOUN" and token.lemma_ not in result  and token.is_stop == False  ]         
        else:
            result.append("")

        return result

    return F.udf(nounLemmatizerMatcher, T.ArrayType(T.StringType()))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


In [48]:
import os
def nounLemmatizeMatcher(df, labelColumn, inputCols, encoding, terms):
    print("tokenize 1")
 
    outputCols = []
    for inputCol in inputCols:

        outputCol = inputCol + "_root_words"

        df = df.withColumn(outputCol, spacy_noun_lemmatize_matcher('en', terms)(F.col(inputCol)))
        print("tokenize 2")
        
        df = df.drop(inputCol)
        print("tokenize 3")

        outputCols.append(outputCol)

    return df, outputCols

In [54]:
!pip install en-pattern-list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement en-pattern-list (from versions: none)
ERROR: No matching distribution found for en-pattern-list


In [55]:
with open('/content/drive/My Drive/AI/en-pattern-list') as f:
    patterns = [trm.rstrip() for trm in f]

In [56]:
patterns

['itching',
 'skin rash',
 'nodal skin eruptions',
 'dischromic  patches',
 'continuous sneezing',
 'shivering chills',
 'watering from eyes',
 'chills',
 'stomach pain',
 'ulcers on tongue',
 'chest pain',
 'yellowish skin',
 'loss of appetite',
 'abdominal pain',
 'yellowing of eyes',
 'passage of gases internal',
 'passage of gases',
 'patches in throat',
 'high fever',
 'indigestion internal',
 'muscle wasting',
 'extra marital contacts',
 'weight loss',
 'irregular sugar level',
 'blurred and distorted vision',
 'excessive hunger',
 'increased appetite',
 'sunken eyes',
 'mucoid sputum',
 'lack of concentration',
 'family history',
 'stiff neck depression',
 'dizziness',
 'headache',
 'breathlessness',
 'cough',
 'fatigue',
 'vomiting',
 'diarrhoea',
 'dehydration',
 'chills',
 'shivering',
 'acidity',
 'nausea',
 'burning micturition',
 'spotting',
 'urination',
 'indigestion',
 'restlessness',
 'lethargy',
 'obesity',
 'irritability',
 'visual disturbances',
 'depression',
 'sti

In [57]:
%%time
tokenized_df, featureColumnNames = nounLemmatizeMatcher(df			= df, 
                                                        labelColumn = labelColumnName, 
                                                        inputCols	= featureColumnNames, 
                                                        encoding	= 'utf-8', 
                                                        terms		= patterns) #TERMS(pattern) should be loaded from file..!!

tokenize 1
tokenize 2
tokenize 3
CPU times: user 14.3 ms, sys: 1.52 ms, total: 15.8 ms
Wall time: 79.6 ms


In [58]:
pandasDF = tokenized_df.toPandas()
compression_opts = dict(method='zip',archive_name='out.csv')  
pandasDF.to_csv('out.zip', index=False, compression=compression_opts) 

In [59]:
pandasDF

,label,text_root_words
0,4.0,"[entertainment, guest, face, recourse]"
1,1.0,"[pdx, schedule]"
2,4.0,"[week, seat, gentleman]"
3,0.0,"[guy, seating, seating, friend, guy, seat, int..."
4,0.0,"[status, match, program, week, response]"
...,...,...
7992,4.0,"[country, plane]"
7993,2.0,"[flight, tomorrow, morning, auto, night, flight]"
7994,1.0,"[cue, delay]"
7995,0.0,"[minute, flight, warning, communication, minut..."
